In [ ]:
import pandas as pd
import numpy as np
import io

In [ ]:
from google.colab import files

# Upload zip file interactively
uploaded = files.upload()

Saving sample_actuals.zip to sample_actuals.zip


In [ ]:
import zipfile
import os

# Assuming the zip file is the first one uploaded
zip_filename = list(uploaded.keys())[0]

# Extract the zip file
with zipfile.ZipFile(io.BytesIO(uploaded[zip_filename]), 'r') as zip_ref:
    zip_ref.extractall('.')

# Assuming there's only one file in the zip or you know the filename
# You might need to adjust this if there are multiple files
extracted_files = os.listdir('.')
data_filename = None
for f in extracted_files:
    if f.endswith('.csv'): # Look for files ending with .csv
        data_filename = f
        break

if data_filename:
    # Load the extracted file into a pandas DataFrame
    try:
        df = pd.read_csv(data_filename)
        print(f"Successfully loaded '{data_filename}' into a DataFrame.")
        display(df.head())
    except Exception as e:
        print(f"Error loading '{data_filename}' into a DataFrame: {e}")
else:
    print("No data file with a .csv extension found in the extracted contents.")

Successfully loaded 'sample_actuals.csv' into a DataFrame.


,ds,location_id,sales_type_id,department_id,y
0,2013-10-29 00:15:00,45093,68413,NaN,0.00
1,2013-10-29 11:30:00,45093,68413,NaN,0.00
2,2013-10-29 12:15:00,45093,68413,NaN,0.00
3,2013-10-29 12:45:00,45093,68413,NaN,39.25
4,2013-10-29 13:00:00,45093,68413,NaN,158.50


In [ ]:
df['ds'] = pd.to_datetime(df['ds'])

# Filter for years 2024, and 2025
df_filtered = df[df['ds'].dt.year.isin([2024])].copy()

# Define the combinations to filter for
filter_combinations = [
    (45093, 45287),
    (121752, 244513),
    (121766, 244513)
]

# Create a boolean mask for the filtering conditions
mask = df_filtered.apply(lambda row: (row['location_id'], row['sales_type_id']) in filter_combinations, axis=1)

# Apply the mask to filter the DataFrame
filtered_actuals_15min = df_filtered[mask].copy()


print("DataFrame filtered for years 2024, and 2025 and the specified combinations:")
display(filtered_actuals_15min.head())

DataFrame filtered for years 2024, and 2025 and the specified combinations:


,ds,location_id,sales_type_id,department_id,y
21516175,2024-01-01 00:00:00,45093,45287,NaN,22.50
22834087,2024-01-04 12:00:00,45093,45287,NaN,42.25
22834088,2024-01-04 13:00:00,45093,45287,NaN,463.50
22834089,2024-01-04 13:45:00,45093,45287,NaN,318.75
22834090,2024-01-04 14:00:00,45093,45287,NaN,80.25


In [ ]:
df['ds'] = pd.to_datetime(df['ds'])

# Filter for years 2024, and 2025
df_filtered_2025 = df[df['ds'].dt.year.isin([2025])].copy()

# Define the combinations to filter for
filter_combinations = [
    (45093, 45287),
    (121752, 244513),
    (121766, 244513)
]

# Create a boolean mask for the filtering conditions
mask = df_filtered_2025.apply(lambda row: (row['location_id'], row['sales_type_id']) in filter_combinations, axis=1)

# Apply the mask to filter the DataFrame
filtered_actuals_15min_2025 = df_filtered_2025[mask].copy()


print("DataFrame filtered for years 2024, and 2025 and the specified combinations:")
display(filtered_actuals_15min_2025.head())

DataFrame filtered for years 2024, and 2025 and the specified combinations:


,ds,location_id,sales_type_id,department_id,y
38859801,2025-01-12 12:15:00,45093,45287,NaN,147.25
38859802,2025-01-12 12:30:00,45093,45287,NaN,70.50
38859803,2025-01-12 13:00:00,45093,45287,NaN,36.25
38859804,2025-01-12 13:15:00,45093,45287,NaN,164.00
38859805,2025-01-12 13:30:00,45093,45287,NaN,111.00


In [ ]:
                                                                                                                                                                                                     combined_actuals_15min = pd.concat([filtered_actuals_15min, filtered_actuals_15min_2025])
print("Combined DataFrame:")
display(combined_actuals_15min.head())
display(combined_actuals_15min.tail())

Combined DataFrame:


,ds,location_id,sales_type_id,department_id,y
21516175,2024-01-01 00:00:00,45093,45287,NaN,22.50
22834087,2024-01-04 12:00:00,45093,45287,NaN,42.25
22834088,2024-01-04 13:00:00,45093,45287,NaN,463.50
22834089,2024-01-04 13:45:00,45093,45287,NaN,318.75
22834090,2024-01-04 14:00:00,45093,45287,NaN,80.25


,ds,location_id,sales_type_id,department_id,y
46771036,2025-07-01 21:00:00,121766,244513,22316.0,7.84
46771037,2025-07-01 21:15:00,121766,244513,22316.0,24.99
46771038,2025-07-01 21:30:00,121766,244513,22316.0,71.78
46771039,2025-07-01 21:45:00,121766,244513,22316.0,41.93
46771040,2025-07-01 22:15:00,121766,244513,22316.0,3.99


In [ ]:
combined_actuals_15min.to_csv('combined_actuals_15min.csv', index=False)
print("DataFrame saved to combined_actuals_15min.csv")

DataFrame saved to combined_actuals_15min.csv


In [ ]:
# Filter for January 1st, 2025 to check the data
jan_1_2025_df = combined_actuals_15min[combined_actuals_15min['ds'].dt.date == pd.to_datetime('2025-01-01').date()].copy()

# Aggregate by sales_type_id and location_id and sum the 'y' column
aggregated_y = jan_1_2025_df.groupby(['location_id', 'sales_type_id'])['y'].sum().reset_index()

print("Aggregated 'y' for various combinations of location_id and sales_type_id on January 1st, 2025:")
display(aggregated_y)

Aggregated 'y' for various combinations of location_id and sales_type_id on January 1st, 2025:


,location_id,sales_type_id,y
0,45093,45287,5936.28
1,121752,244513,3242.80
2,121766,244513,4401.48


In [ ]:
# =========================================
# STEP 0: Imports
# =========================================
from google.colab import files
import pandas as pd
import numpy as np

# =========================================
# STEP 1: Upload Files
# =========================================
print("📂 Please upload these files:")
print("1. daily_forecast_2025.csv (columns: location_id, date, y_pred)")
print("2. actuals_15min.csv (columns: location_id, ds_x, y)")
print("3. locations.csv (columns: id, corporation_id, postal_code)")
print("4. weather_hourly.csv (columns: id, country_code, ds, postal_code, real_feel, precipitation, snow, coverage, weather_code, icon)")


uploaded = files.upload()  # <-- Upload all four files here

# Read daily_forecast with the correct date format
daily_forecast = pd.read_csv(
    "forecast_2025_xgb_final.csv",
    low_memory=False,
    parse_dates=['date'],
    dayfirst=True  # Specify that the day comes first (DD-MM-YYYY)
)
# actuals_15min_2024 = pd.read_csv("aggregated_15min_df_2024.csv",low_memory=False, parse_dates=['ds'],dayfirst=True)
# actuals_15min_2025 = pd.read_csv("aggregated_15min_df_2025.csv",low_memory=False, parse_dates=['ds'],dayfirst=True)
locations = pd.read_csv("locations.csv")
weather = pd.read_csv("weather_hourly.csv", low_memory=False, parse_dates=['ds'],dayfirst=True)


print("\n✅ Files uploaded and loaded successfully.")
display(daily_forecast.head())
display(locations.head())
display(weather.head())

📂 Please upload these files:
1. daily_forecast_2025.csv (columns: location_id, date, y_pred)
2. actuals_15min.csv (columns: location_id, ds_x, y)
3. locations.csv (columns: id, corporation_id, postal_code)
4. weather_hourly.csv (columns: id, country_code, ds, postal_code, real_feel, precipitation, snow, coverage, weather_code, icon)


Saving forecast_2025_xgb_final.csv to forecast_2025_xgb_final.csv
Saving locations.csv to locations.csv
Saving weather_hourly.csv to weather_hourly.csv

✅ Files uploaded and loaded successfully.


/tmp/ipython-input-3145328710.py:30: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  weather = pd.read_csv("weather_hourly.csv", low_memory=False, parse_dates=['ds'],dayfirst=True)


,date,location_id,sales_type_id,y_daily,holiday,is_holiday,industry,real_feel,precipitation,snow,...,offset_days,is_holiday_halo,halo_offset_days,day_of_month,week_of_year,quarter,month,week_of_month,day_of_week_numeric,y_pred
0,2025-01-01,121752,244513,3242.80,10,1,1,3.693750,0.00,0.000,...,0,1,0,1,1,1,1,1,2,3688.9856
1,2025-01-02,121752,244513,3511.74,-1,0,1,6.071667,0.00,0.000,...,0,0,0,2,1,1,1,1,3,4099.4067
2,2025-01-03,121752,244513,5262.18,-1,0,1,3.169167,0.00,0.000,...,0,0,0,3,1,1,1,1,4,5505.9155
3,2025-01-04,121752,244513,3813.45,-1,0,1,6.986667,0.00,0.000,...,0,0,0,4,1,1,1,1,5,4644.6436
4,2025-01-05,121752,244513,3104.90,-1,0,1,-1.806250,1.71,0.472,...,0,0,0,5,1,1,1,1,6,3927.1543


,id,corporation_id,postal_code
0,144484,91906,66209
1,144391,91906,66220
2,144818,92250,92651
3,144789,92250,92618
4,144338,91908,10002


In [ ]:
actuals_15min = combined_actuals_15min.copy()

In [ ]:
display(actuals_15min.head())

,ds,location_id,sales_type_id,department_id,y
21516175,2024-01-01 00:00:00,45093,45287,NaN,22.50
22834087,2024-01-04 12:00:00,45093,45287,NaN,42.25
22834088,2024-01-04 13:00:00,45093,45287,NaN,463.50
22834089,2024-01-04 13:45:00,45093,45287,NaN,318.75
22834090,2024-01-04 14:00:00,45093,45287,NaN,80.25


In [ ]:
# Add postal_code and corporation_id from locations
actuals_15min = actuals_15min.merge(
    locations[['id', 'corporation_id', 'postal_code']],
    left_on='location_id', right_on='id', how='left'
).drop(columns='id')

print("\n✅ Added postal_code and corporation_id to actuals_15min.")
display(actuals_15min.head())


✅ Added postal_code and corporation_id to actuals_15min.


,ds,location_id,sales_type_id,department_id,y,corporation_id,postal_code
0,2024-01-01 00:00:00,45093,45287,NaN,22.50,46113,T7Z 2W7
1,2024-01-04 12:00:00,45093,45287,NaN,42.25,46113,T7Z 2W7
2,2024-01-04 13:00:00,45093,45287,NaN,463.50,46113,T7Z 2W7
3,2024-01-04 13:45:00,45093,45287,NaN,318.75,46113,T7Z 2W7
4,2024-01-04 14:00:00,45093,45287,NaN,80.25,46113,T7Z 2W7


In [ ]:
# Ensure 'ds' is datetime before finding min/max
actuals_15min['ds'] = pd.to_datetime(actuals_15min['ds'])

# Create a list of unique location_id, sales_type_id, and department_id combinations
unique_combinations = actuals_15min[['location_id', 'sales_type_id', 'department_id', 'corporation_id', 'postal_code']].drop_duplicates()

# Determine the minimum and maximum dates from the actuals data
min_date = actuals_15min['ds'].min().date()
max_date = actuals_15min['ds'].max().date()

# Generate a complete date range with a 15-minute frequency
all_datetimes = pd.date_range(start=min_date, end=max_date + pd.Timedelta(days=1) - pd.Timedelta(minutes=15), freq='15min')

# Create a complete time series DataFrame
complete_ts = pd.DataFrame()

for index, row in unique_combinations.iterrows():
    location_id = row['location_id']
    sales_type_id = row['sales_type_id']
    department_id = row['department_id']
    corporation_id = row['corporation_id']
    postal_code = row['postal_code']


    temp_df = pd.DataFrame({'ds': all_datetimes})
    temp_df['location_id'] = location_id
    temp_df['sales_type_id'] = sales_type_id
    temp_df['department_id'] = department_id
    temp_df['corporation_id'] = corporation_id
    temp_df['postal_code'] = postal_code


    complete_ts = pd.concat([complete_ts, temp_df], ignore_index=True)

# Merge the complete time series with the actuals data
# Use an outer merge to keep all time slots from complete_ts and fill missing values with 0
actuals_with_missing = pd.merge(
    complete_ts,
    actuals_15min,
    on=['ds', 'location_id', 'sales_type_id', 'department_id', 'corporation_id', 'postal_code'],
    how='left'
)

# Fill NaN values in the 'y' column with 0 (for missing time slots)
actuals_with_missing['y'] = actuals_with_missing['y'].fillna(0)

# Display the head and tail of the new DataFrame to show filled missing values
print("DataFrame with missing 15-minute slots filled:")
display(actuals_with_missing.head())
display(actuals_with_missing.tail())

DataFrame with missing 15-minute slots filled:


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y
0,2024-01-01 00:00:00,45093,45287,NaN,46113,T7Z 2W7,22.5
1,2024-01-01 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0
2,2024-01-01 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0
3,2024-01-01 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.0
4,2024-01-01 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y
368930,2025-07-02 23:00:00,121766,244513,22316.0,78415,74429,0.0
368931,2025-07-02 23:15:00,121766,244513,22316.0,78415,74429,0.0
368932,2025-07-02 23:30:00,121766,244513,22316.0,78415,74429,0.0
368933,2025-07-02 23:45:00,121766,244513,22316.0,78415,74429,0.0
368934,2025-07-03 00:00:00,121766,244513,22316.0,78415,74429,0.0


In [ ]:
# Check the number of unique combinations in the DataFrame with missing values filled
num_unique_combinations_filled = actuals_with_missing[['location_id', 'sales_type_id', 'department_id']].drop_duplicates().shape[0]
print(f"Number of unique combinations in the DataFrame with missing slots filled: {num_unique_combinations_filled}")
# You can also display the unique combinations to verify
print("\nUnique combinations in the filled DataFrame:")
display(actuals_with_missing[['location_id', 'sales_type_id', 'department_id']].drop_duplicates())

Number of unique combinations in the DataFrame with missing slots filled: 7

Unique combinations in the filled DataFrame:


,location_id,sales_type_id,department_id
0,45093,45287,NaN
52705,121752,244513,22316.0
105410,121752,244513,22313.0
158115,121752,244513,22315.0
210820,121766,244513,22313.0
263525,121766,244513,22315.0
316230,121766,244513,22316.0


In [ ]:
# Save the actuals_15min DataFrame to a CSV file
actuals_with_missing.to_csv('actuals_15min_filled_intervals.csv', index=False)
print("Actuals 15min DataFrame with filled intervals saved to 'actuals_15min_filled_intervals.csv'.")

Actuals 15min DataFrame with filled intervals saved to 'actuals_15min_filled_intervals.csv'.


In [ ]:
actuals_15min=actuals_with_missing.copy()

In [ ]:
# Filter for location_id = 45093 and January 1st, 2025
location_45093_jan_1_2025 = actuals_with_missing[
    (actuals_with_missing['location_id'] == 45093) &
    (actuals_with_missing['ds'].dt.date == pd.to_datetime('2024-01-01').date())
].copy()

print("Time series data for location_id = 45093 on January 1st, 2025:")
display(location_45093_jan_1_2025)

Time series data for location_id = 45093 on January 1st, 2025:


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y
0,2024-01-01 00:00:00,45093,45287,NaN,46113,T7Z 2W7,22.50
1,2024-01-01 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.00
2,2024-01-01 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.00
3,2024-01-01 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.00
4,2024-01-01 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.00
...,...,...,...,...,...,...,...
91,2024-01-01 22:45:00,45093,45287,NaN,46113,T7Z 2W7,0.00
92,2024-01-01 23:00:00,45093,45287,NaN,46113,T7Z 2W7,0.00
93,2024-01-01 23:15:00,45093,45287,NaN,46113,T7Z 2W7,27.25
94,2024-01-01 23:30:00,45093,45287,NaN,46113,T7Z 2W7,0.00


In [ ]:
# Filter for location_id = 45093 and January 1st, 2025
location_45093_jan_1_2025_agg = actuals_with_missing[
    (actuals_with_missing['location_id'] == 45093) &
    (actuals_with_missing['ds'].dt.date == pd.to_datetime('2025-01-01').date())
].copy()

# Calculate the sum of 'y' for this filtered data
aggregated_y_45093_jan_1_2025 = location_45093_jan_1_2025_agg['y'].sum()

print(f"Aggregated 'y' for location_id = 45093 on January 1st, 2025: {aggregated_y_45093_jan_1_2025}")

Aggregated 'y' for location_id = 45093 on January 1st, 2025: 5936.28


In [ ]:
# Check for missing values in each column
missing_values = actuals_15min.isnull().sum()

# Print the columns with missing values
print("Missing values per column in actuals_15min:")
print(missing_values[missing_values > 0])

Missing values per column in actuals_15min:
department_id    52705
dtype: int64


In [ ]:
# Ensure postal codes are string and lowercase, and remove spaces in actuals_15min
actuals_15min['postal_code_cleaned'] = actuals_15min['postal_code'].astype(str).str.lower().str.replace(' ', '', regex=False)

# Ensure postal codes are string and lowercase, and remove spaces in weather
weather['postal_code_cleaned'] = weather['postal_code'].astype(str).str.lower().str.replace(' ', '', regex=False)

# Get distinct postal codes from actuals_15min
distinct_postal_codes_actuals = actuals_15min['postal_code_cleaned'].unique()

# Get distinct postal codes from weather
distinct_postal_codes_weather = weather['postal_code_cleaned'].unique()

print("Distinct postal codes in actuals_15min after cleaning:")
print(distinct_postal_codes_actuals)

print("\nDistinct postal codes in weather after cleaning:")
print(distinct_postal_codes_weather)

# You can optionally compare the sets of postal codes to see if there are any discrepancies
# print("\nPostal codes in actuals_15min not in weather:")
# print(set(distinct_postal_codes_actuals) - set(distinct_postal_codes_weather))

# print("\nPostal codes in weather not in actuals_15min:")
# print(set(distinct_postal_codes_weather) - set(distinct_postal_codes_actuals))

# Drop the temporary cleaned columns if you don't need them after verification
# actuals_15min = actuals_15min.drop(columns='postal_code_cleaned')
# weather = weather.drop(columns='postal_code_cleaned')

Distinct postal codes in actuals_15min after cleaning:
['t7z2w7' '74063' '74429']

Distinct postal codes in weather after cleaning:
['33156' '33186' '10002' '75056' '76401' '66220' '66209' '74063' '74429'
 '63383' '65803' '65810' '65079' '65049' '64772' '92651' 't7z2w7' 't9m1p2'
 '92618' '89135']


In [ ]:
# Ensure 'ds' in weather is datetime
weather['ds'] = pd.to_datetime(weather['ds'])
# Create an hourly datetime column for merging
weather['hourly_datetime'] = weather['ds'].dt.floor('h')

# Ensure 'ds' in actuals_15min is datetime
actuals_15min['ds'] = pd.to_datetime(actuals_15min['ds'])
# Create an hourly datetime column for merging from the 'ds' column
actuals_15min['hourly_datetime'] = actuals_15min['ds'].dt.floor('h')


# We have already cleaned postal codes in the previous step, so we can use the cleaned column for merging.
# If the cleaned column was not kept, you would re-apply the cleaning here:
# actuals_15min['postal_code_cleaned'] = actuals_15min['postal_code'].astype(str).str.lower().str.replace(' ', '', regex=False)
# weather['postal_code_cleaned'] = weather['postal_code'].astype(str).str.lower().str.replace(' ', '', regex=False)


# Select weather columns to merge from weather, using the cleaned postal code column
weather_cols_to_merge = ['postal_code_cleaned', 'hourly_datetime', 'real_feel', 'precipitation', 'snow', 'coverage', 'weather_code']

# Perform a left merge on postal_code_cleaned and the hourly datetime
merged_df = pd.merge(
    actuals_15min,
    weather[weather_cols_to_merge],
    left_on=['postal_code_cleaned', 'hourly_datetime'],
    right_on=['postal_code_cleaned', 'hourly_datetime'],
    how='left'
)

# Drop the helper column (hourly_datetime) and the cleaned postal code column if you don't need them after verification
merged_df = merged_df.drop(columns=['hourly_datetime', 'postal_code_cleaned'])


# Display shape and head of the merged DataFrame
print("Shape of the merged DataFrame:", merged_df.shape)
display(merged_df.head())

# Update the actuals_15min dataframe to the merged_df
actuals_15min = merged_df

Shape of the merged DataFrame: (368935, 12)


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y,real_feel,precipitation,snow,coverage,weather_code
0,2024-01-01 00:00:00,45093,45287,NaN,46113,T7Z 2W7,22.5,0.71,0.0,0.0,73.0,::BK
1,2024-01-01 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK
2,2024-01-01 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK
3,2024-01-01 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK
4,2024-01-01 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.49,0.0,0.0,0.0,::CL


In [ ]:
# Check for missing values in each column
missing_values = actuals_15min.isnull().sum()

# Print the columns with missing values
print("Missing values per column in actuals_15min:")
print(missing_values[missing_values > 0])

Missing values per column in actuals_15min:
department_id    52705
real_feel           56
precipitation       56
snow                56
coverage            56
weather_code        56
dtype: int64


In [ ]:
print("Number of rows in actuals_15min:", actuals_15min.shape[0])

Number of rows in actuals_15min: 368935


In [ ]:
# Ensure 'ds' is in datetime format
weather['ds'] = pd.to_datetime(weather['ds'], errors='coerce')

# Filter weather data for the specific date
weather_march_10_2024 = weather[weather['ds'].dt.date == pd.to_datetime('2024-03-10').date()].copy()

# Display the filtered data
print("Weather data for March 10, 2024:")
display(weather_march_10_2024)

# Filter weather data for postal code 74063 on March 10, 2024
weather_74063_march_10 = weather_march_10_2024[weather_march_10_2024['postal_code_cleaned'] == '74063'].copy()

# Display the filtered data
print("Weather data for postal code 74063 on March 10, 2024:")
display(weather_74063_march_10)

Weather data for March 10, 2024:


,id,country_code,ds,postal_code,real_feel,precipitation,snow,coverage,weather_code,icon,postal_code_cleaned,hourly_datetime
17437,360191,1,2024-03-10 23:00:00,33156,24.71,0.0,0.0,18,::FW,NaN,33156,2024-03-10 23:00:00
18532,361716,1,2024-03-10 23:00:00,33186,24.71,0.0,0.0,18,::FW,NaN,33186,2024-03-10 23:00:00
302595,883777,1,2024-03-10 00:00:00,76401,5.76,0.0,0.0,1,::CL,NaN,76401,2024-03-10 00:00:00
304776,897455,1,2024-03-10 00:00:00,66220,3.76,0.0,0.0,0,::CL,NaN,66220,2024-03-10 00:00:00
354432,700033,1,2024-03-10 01:00:00,75056,6.26,0.0,0.0,1,::CL,NaN,75056,2024-03-10 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
521722,16683782,1,2024-03-10 19:00:00,89135,20.45,0.0,0.0,6,::CL,NaN,89135,2024-03-10 19:00:00
521723,16683783,1,2024-03-10 20:00:00,89135,19.85,0.0,0.0,5,::CL,NaN,89135,2024-03-10 20:00:00
521724,16683784,1,2024-03-10 21:00:00,89135,18.97,0.0,0.0,6,::CL,NaN,89135,2024-03-10 21:00:00
521725,16683785,1,2024-03-10 22:00:00,89135,18.48,0.0,0.0,6,::CL,NaN,89135,2024-03-10 22:00:00


Weather data for postal code 74063 on March 10, 2024:


,id,country_code,ds,postal_code,real_feel,precipitation,snow,coverage,weather_code,icon,postal_code_cleaned,hourly_datetime
355139,4376652,1,2024-03-10 00:00:00,74063,5.41,0.0,0.0,0,::CL,NaN,74063,2024-03-10 00:00:00
479953,4376653,1,2024-03-10 01:00:00,74063,5.73,0.0,0.0,0,::CL,NaN,74063,2024-03-10 01:00:00
479954,4376654,1,2024-03-10 03:00:00,74063,5.43,0.0,0.0,0,::CL,NaN,74063,2024-03-10 03:00:00
479955,4376655,1,2024-03-10 04:00:00,74063,4.39,0.0,0.0,0,::CL,NaN,74063,2024-03-10 04:00:00
479956,4376656,1,2024-03-10 05:00:00,74063,3.86,0.0,0.0,0,::CL,NaN,74063,2024-03-10 05:00:00
479957,4376657,1,2024-03-10 06:00:00,74063,4.32,0.0,0.0,0,::CL,NaN,74063,2024-03-10 06:00:00
479958,4376658,1,2024-03-10 07:00:00,74063,4.79,0.0,0.0,0,::CL,NaN,74063,2024-03-10 07:00:00
479959,4376659,1,2024-03-10 08:00:00,74063,4.37,0.0,0.0,0,::CL,NaN,74063,2024-03-10 08:00:00
492706,4376660,1,2024-03-10 09:00:00,74063,6.96,0.0,0.0,0,::CL,NaN,74063,2024-03-10 09:00:00
492707,4376661,1,2024-03-10 10:00:00,74063,10.32,0.0,0.0,0,::CL,NaN,74063,2024-03-10 10:00:00


In [ ]:
#### Checks

In [ ]:
# Ensure 'ds' is in datetime format for filtering
weather['ds'] = pd.to_datetime(weather['ds'], errors='coerce')

# Filter weather data for the specific date
weather_august_9_2025 = weather[weather['ds'].dt.date == pd.to_datetime('2025-08-09').date()].copy()

# Display the filtered data
print("Weather data for August 9, 2025:")
display(weather_august_9_2025)

Weather data for August 9, 2025:


,id,country_code,ds,postal_code,real_feel,precipitation,snow,coverage,weather_code,icon,postal_code_cleaned,hourly_datetime


In [ ]:
display(actuals_15min.head())

,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y,real_feel,precipitation,snow,coverage,weather_code
0,2024-01-01 00:00:00,45093,45287,NaN,46113,T7Z 2W7,22.5,0.71,0.0,0.0,73.0,::BK
1,2024-01-01 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK
2,2024-01-01 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK
3,2024-01-01 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK
4,2024-01-01 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.49,0.0,0.0,0.0,::CL


In [ ]:
### Checks

In [ ]:
# Filter for location_id = 45093 and January 1st, 2025
location_45093_jan_1_2025 = actuals_with_missing[
    (actuals_with_missing['location_id'] == 45093) &
    (actuals_with_missing['ds'].dt.date == pd.to_datetime('2025-01-01').date())
].copy()

print("Time series data for location_id = 45093 on January 1st, 2025:")
display(location_45093_jan_1_2025)

Time series data for location_id = 45093 on January 1st, 2025:


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y
35136,2025-01-01 00:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0
35137,2025-01-01 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0
35138,2025-01-01 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0
35139,2025-01-01 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.0
35140,2025-01-01 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0
...,...,...,...,...,...,...,...
35227,2025-01-01 22:45:00,45093,45287,NaN,46113,T7Z 2W7,50.0
35228,2025-01-01 23:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0
35229,2025-01-01 23:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0
35230,2025-01-01 23:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0


In [ ]:
from google.colab import files
import pandas as pd

# 📥 Upload the file interactively
uploaded = files.upload()

# 🔁 Extract filename automatically
filename = list(uploaded.keys())[0]

# 📄 Load the file into a DataFrame
holiday = pd.read_csv(filename)

# 👀 Preview
print(f"✅ Uploaded and loaded {filename} into 'holiday'")
holiday.head()

Saving holiday.csv to holiday.csv
✅ Uploaded and loaded holiday.csv into 'holiday'


,corporation_id,ds,holiday,lower_window,upper_window
0,78415,2010-02-07,Superbowl,0,0
1,78415,2011-02-06,Superbowl,0,0
2,78415,2012-02-05,Superbowl,0,0
3,78415,2013-02-03,Superbowl,0,0
4,78415,2014-02-02,Superbowl,0,0


In [ ]:
# Ensure 'corporation_id' in both dataframes are of the same type before merging
# Check the data types
print(f"Data type of 'corporation_id' in actuals_15min: {actuals_15min['corporation_id'].dtype}")
print(f"Data type of 'corporation_id' in holiday: {holiday['corporation_id'].dtype}")

# Convert 'corporation_id' in actuals_15min to int64 if it's float
if actuals_15min['corporation_id'].dtype == 'float64':
    actuals_15min['corporation_id'] = actuals_15min['corporation_id'].astype('int64', errors='ignore')

# Convert 'corporation_id' in holiday to int64 if it's not already
if holiday['corporation_id'].dtype != 'int64':
    holiday['corporation_id'] = holiday['corporation_id'].astype('int64', errors='ignore')

# Ensure the date column in holiday is in datetime format and rename it for merging
holiday['ds'] = pd.to_datetime(holiday['ds'], errors='coerce')
holiday_to_merge = holiday.rename(columns={'ds': 'date'})


# Ensure the ds column in actuals_15min is in datetime format and extract the date part
actuals_15min['ds'] = pd.to_datetime(actuals_15min['ds'])
actuals_15min['date'] = actuals_15min['ds'].dt.date # Extract date part

# Convert the 'date' column in actuals_15min to datetime64[ns]
actuals_15min['date'] = pd.to_datetime(actuals_15min['date'])

# Perform the merge on 'corporation_id' and 'date'
merged_df = pd.merge(
    actuals_15min,
    holiday_to_merge[['corporation_id','date', 'holiday']],
    on=['corporation_id', 'date'],
    how='left'
)

# Update the actuals_15min dataframe to the merged_df
actuals_15min = merged_df

print("\nHoliday data merged successfully based on corporation_id and date.")
display(actuals_15min.head())

Data type of 'corporation_id' in actuals_15min: int64
Data type of 'corporation_id' in holiday: int64

Holiday data merged successfully based on corporation_id and date.


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y,real_feel,precipitation,snow,coverage,weather_code,date,holiday
0,2024-01-01 00:00:00,45093,45287,NaN,46113,T7Z 2W7,22.5,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day
1,2024-01-01 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day
2,2024-01-01 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day
3,2024-01-01 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day
4,2024-01-01 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.49,0.0,0.0,0.0,::CL,2024-01-01,New Year's Day


In [ ]:
# Filter for rows where 'holiday' is not NaN and select the relevant columns
holiday_combinations = actuals_15min[actuals_15min['holiday'].notna()][['date', 'holiday', 'corporation_id', 'sales_type_id']].drop_duplicates()

# Print the distinct date, holiday, corporation ID, and sales type ID combinations
print("Distinct combinations of date, holiday, corporation ID, and sales type ID:")
display(holiday_combinations)

Distinct combinations of date, holiday, corporation ID, and sales type ID:


,date,holiday,corporation_id,sales_type_id
0,2024-01-01,New Year's Day,46113,45287
3936,2024-02-11,Superbowl,46113,45287
4224,2024-02-14,Valentines Day,46113,45287
8640,2024-03-31,Easter Sunday,46113,45287
12672,2024-05-12,Mother's Day,46113,45287
14112,2024-05-27,Memorial Day,46113,45287
17760,2024-07-04,Canada Day,46113,45287
23520,2024-09-02,Labor Day,46113,45287
27552,2024-10-14,Thanksgiving Day,46113,45287
29184,2024-10-31,Halloween,46113,45287


In [ ]:
### Merge industry data

from google.colab import files
import pandas as pd

# 📥 Upload the file interactively
uploaded = files.upload()

# 🔁 Extract filename automatically
filename = list(uploaded.keys())[0]

# 📄 Load the file into a DataFrame
industry = pd.read_csv(filename)

# 👀 Preview
print(f"✅ Uploaded and loaded {filename} into 'industry'")
industry.head()

Saving industry_classification.csv to industry_classification.csv
✅ Uploaded and loaded industry_classification.csv into 'industry'


,corporation_id,industry
0,46113,restaurant
1,78415,pizza chain
2,89766,grocery
3,90816,grocery
4,91133,restaurant


In [ ]:
df=actuals_15min.copy()

In [ ]:
print("Number of rows in actuals_15min:", df.shape[0])

Number of rows in actuals_15min: 368935


In [ ]:
# Merge industry data
df = df.merge(
    industry[['corporation_id', 'industry']],
    on='corporation_id',
    how='left'
)

print("Merged industry data with daily_sales:")
display(df.head())

Merged industry data with daily_sales:


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y,real_feel,precipitation,snow,coverage,weather_code,date,holiday,industry
0,2024-01-01 00:00:00,45093,45287,NaN,46113,T7Z 2W7,22.5,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant
1,2024-01-01 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant
2,2024-01-01 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant
3,2024-01-01 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant
4,2024-01-01 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.49,0.0,0.0,0.0,::CL,2024-01-01,New Year's Day,restaurant


In [ ]:
from google.colab import files
import pandas as pd

# 📥 Upload the file interactively
uploaded = files.upload()

# 🔁 Extract filename automatically
filename = list(uploaded.keys())[0]

# 📄 Load the file into a DataFrame
location_hours = pd.read_csv(filename)

# 👀 Preview
print(f"✅ Uploaded and loaded {filename} into 'location_hours'")
location_hours.head()

Saving location_hours_cleaned.csv to location_hours_cleaned.csv
✅ Uploaded and loaded location_hours_cleaned.csv into 'location_hours'


,location_id,day,opening_time,closing_time
0,144370,Sunday,12:00,00:00
1,144370,Monday,12:00,00:00
2,144370,Tuesday,12:00,00:00
3,144370,Wednesday,12:00,00:00
4,144370,Thursday,12:00,01:00


In [ ]:
print(location_hours.columns.tolist())

['location_id', 'day', 'opening_time', 'closing_time']


In [ ]:
print("Merged dtypes:")
print(df.dtypes)
print("\nLocation_hours dtypes:")
print(location_hours.dtypes)

print("\nUnique values check:")
print("Merged location_ids:", df['location_id'].unique())
print("Location_hours location_ids:", location_hours['location_id'].unique())


Merged dtypes:
ds                datetime64[ns]
location_id                int64
sales_type_id              int64
department_id            float64
corporation_id             int64
postal_code               object
y                        float64
real_feel                float64
precipitation            float64
snow                     float64
coverage                 float64
weather_code              object
date              datetime64[ns]
holiday                   object
industry                  object
dtype: object

Location_hours dtypes:
location_id      int64
day             object
opening_time    object
closing_time    object
dtype: object

Unique values check:
Merged location_ids: [ 45093 121752 121766]
Location_hours location_ids: [144370 144338 144679 144683 143099 143027 121752 121766 144391 144484
 135145  45093 144249 142228 142233 144818 140751 140755 140749 144789]


In [ ]:
import pandas as pd

# 1. Extract day of week and time from merged data
df['day'] = df['ds'].dt.day_name()   # e.g., 'Monday'

# Extract the time part from the 'ds' column
df['time'] = df['ds'].dt.time

# Now df has 'day' and 'time' columns extracted from 'ds'
print("DataFrame with 'day' and 'time' columns:")
display(df.head())

DataFrame with 'day' and 'time' columns:


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y,real_feel,precipitation,snow,coverage,weather_code,date,holiday,industry,day,time
0,2024-01-01 00:00:00,45093,45287,NaN,46113,T7Z 2W7,22.5,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant,Monday,00:00:00
1,2024-01-01 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant,Monday,00:15:00
2,2024-01-01 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant,Monday,00:30:00
3,2024-01-01 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant,Monday,00:45:00
4,2024-01-01 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.49,0.0,0.0,0.0,::CL,2024-01-01,New Year's Day,restaurant,Monday,01:00:00


In [ ]:
# 2. Convert opening/closing times to datetime.time
location_hours['opening_time'] = pd.to_datetime(location_hours['opening_time'], format='%H:%M').dt.time
location_hours['closing_time'] = pd.to_datetime(location_hours['closing_time'], format='%H:%M').dt.time

In [ ]:
print("time dtype:", type(df['time'].iloc[0]))
print("opening_time dtype:", type(location_hours['opening_time'].iloc[0]))
print("closing_time dtype:", type(location_hours['closing_time'].iloc[0]))

time dtype: <class 'datetime.time'>
opening_time dtype: <class 'datetime.time'>
closing_time dtype: <class 'datetime.time'>


In [ ]:
print("Distinct day values in merged:")
print(df['day'].dropna().unique())

print("\nDistinct day values in location_hours:")
print(location_hours['day'].dropna().unique())

Distinct day values in merged:
['Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday' 'Saturday' 'Sunday']

Distinct day values in location_hours:
['Sunday' 'Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday' 'Saturday'
 'Monday ' 'Friday ' 'Saturday ']


In [ ]:
# Strip leading/trailing spaces and standardize capitalization
location_hours['day'] = location_hours['day'].str.strip().str.capitalize()

# Double-check after cleaning
print("Distinct day values in location_hours (cleaned):")
print(location_hours['day'].unique())


Distinct day values in location_hours (cleaned):
['Sunday' 'Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday' 'Saturday']


In [ ]:

df_2=df.copy()

# 3. Merge on location_id + day
merged_new = df.merge(
    location_hours,
    on=['location_id', 'day'],
    how='left'
)

In [ ]:
from datetime import time

def check_open(row):
    if pd.isnull(row['opening_time']) or pd.isnull(row['closing_time']):
        return 0

    t = row['time'] # Corrected column name from 'time_x' to 'time'
    open_t = row['opening_time']
    close_t = row['closing_time']

    # Case 1: Closing at 00:00 means store closes end of day (23:59:59)
    if close_t == time(0, 0):
        return int(open_t <= t <= time(23, 59, 59))

    # Case 2: Overnight hours (e.g. open 22:00, close 06:00)
    if open_t > close_t:
        return int(t >= open_t or t <= close_t)

    # Case 3: Normal same-day hours
    return int(open_t <= t <= close_t)

merged_new['is_open'] = merged_new.apply(check_open, axis=1)

display(merged_new[['location_id','ds','time','day','opening_time','closing_time','is_open']].head(10)) # Corrected column name

,location_id,ds,time,day,opening_time,closing_time,is_open
0,45093,2024-01-01 00:00:00,00:00:00,Monday,11:00:00,00:00:00,0
1,45093,2024-01-01 00:15:00,00:15:00,Monday,11:00:00,00:00:00,0
2,45093,2024-01-01 00:30:00,00:30:00,Monday,11:00:00,00:00:00,0
3,45093,2024-01-01 00:45:00,00:45:00,Monday,11:00:00,00:00:00,0
4,45093,2024-01-01 01:00:00,01:00:00,Monday,11:00:00,00:00:00,0
5,45093,2024-01-01 01:15:00,01:15:00,Monday,11:00:00,00:00:00,0
6,45093,2024-01-01 01:30:00,01:30:00,Monday,11:00:00,00:00:00,0
7,45093,2024-01-01 01:45:00,01:45:00,Monday,11:00:00,00:00:00,0
8,45093,2024-01-01 02:00:00,02:00:00,Monday,11:00:00,00:00:00,0
9,45093,2024-01-01 02:15:00,02:15:00,Monday,11:00:00,00:00:00,0


In [ ]:
merged_new.to_csv('merged_new.csv', index=False)
print("daily_sales DataFrame saved to 'merged_new.csv'")

daily_sales DataFrame saved to 'merged_new.csv'


In [ ]:
display(merged_new.head(10))

,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y,real_feel,precipitation,snow,coverage,weather_code,date,holiday,industry,day,time,opening_time,closing_time,is_open
0,2024-01-01 00:00:00,45093,45287,NaN,46113,T7Z 2W7,22.5,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant,Monday,00:00:00,11:00:00,00:00:00,0
1,2024-01-01 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant,Monday,00:15:00,11:00:00,00:00:00,0
2,2024-01-01 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant,Monday,00:30:00,11:00:00,00:00:00,0
3,2024-01-01 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,73.0,::BK,2024-01-01,New Year's Day,restaurant,Monday,00:45:00,11:00:00,00:00:00,0
4,2024-01-01 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.49,0.0,0.0,0.0,::CL,2024-01-01,New Year's Day,restaurant,Monday,01:00:00,11:00:00,00:00:00,0
5,2024-01-01 01:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.49,0.0,0.0,0.0,::CL,2024-01-01,New Year's Day,restaurant,Monday,01:15:00,11:00:00,00:00:00,0
6,2024-01-01 01:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.49,0.0,0.0,0.0,::CL,2024-01-01,New Year's Day,restaurant,Monday,01:30:00,11:00:00,00:00:00,0
7,2024-01-01 01:45:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.49,0.0,0.0,0.0,::CL,2024-01-01,New Year's Day,restaurant,Monday,01:45:00,11:00:00,00:00:00,0
8,2024-01-01 02:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-1.77,0.0,0.0,0.0,::CL,2024-01-01,New Year's Day,restaurant,Monday,02:00:00,11:00:00,00:00:00,0
9,2024-01-01 02:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-1.77,0.0,0.0,0.0,::CL,2024-01-01,New Year's Day,restaurant,Monday,02:15:00,11:00:00,00:00:00,0


In [ ]:
merged_new.to_csv('merged_new.csv', index=False)
print("daily_sales DataFrame saved to 'merged_new.csv'")

daily_sales DataFrame saved to 'merged_new.csv'


In [ ]:
import pandas as pd
from google.colab import files

# Upload the file
uploaded = files.upload()

# Load the correct file (based on the uploaded name)
holiday_halo_detail_final = pd.read_csv("holiday_halo_detail_final.csv", parse_dates=['holiday_date'])

holiday_halo_detail_final.head()

Saving holiday_halo_detail_final.csv to holiday_halo_detail_final.csv


,location_id,sales_type_id,holiday_date,holiday,before_days,after_days
0,45093,45287,01-01-2024,New Year's Day,1,3
1,45093,45287,11-02-2024,Superbowl,0,1
2,45093,45287,14-02-2024,Valentines Day,0,0
3,45093,45287,31-03-2024,Easter Sunday,1,0
4,45093,45287,12-05-2024,Mother's Day,0,0


In [ ]:
def expand_holiday_halo(holiday_halo_detail: pd.DataFrame) -> pd.DataFrame:
    """
    Expand each holiday row into rows for:
      - the holiday itself (offset_days = 0, halo_type = 'holiday')
      - before_days (offset_days = -k, halo_type = f'pre_{k}')
      - after_days  (offset_days = +k, halo_type = f'post_{k}')
    Only the holiday itself carries the holiday name; halo rows reference it via `halo_of`.
    """
    recs = []
    for _, row in holiday_halo_detail.iterrows():
        # Explicit format avoids ambiguity (DD-MM-YYYY)
        base_date = pd.to_datetime(row['holiday_date'], format="%d-%m-%Y")
        loc, stype, holiday_name = row['location_id'], row['sales_type_id'], row['holiday']

        # Holiday itself (true holiday row)
        recs.append((loc, stype, base_date, holiday_name, 'holiday', 0, None))

        # Before days (halo rows)
        for k in range(1, int(row['before_days']) + 1):
            recs.append((loc, stype,
                         base_date - pd.Timedelta(days=k),
                         None, f'pre_{k}', -k, holiday_name))

        # After days (halo rows)
        for k in range(1, int(row['after_days']) + 1):
            recs.append((loc, stype,
                         base_date + pd.Timedelta(days=k),
                         None, f'post_{k}', k, holiday_name))

    return pd.DataFrame(
        recs,
        columns=['location_id', 'sales_type_id', 'date',
                 'holiday', 'halo_type', 'offset_days', 'halo_of']
    )


In [ ]:
holiday_halo_expanded = expand_holiday_halo(holiday_halo_detail_final)

In [ ]:
holiday_before_halo=merged_new.copy()

In [ ]:
# Ensure 'ds' column is datetime
holiday_halo_expanded['date'] = pd.to_datetime(holiday_halo_expanded['date'])
merged_new['date'] = pd.to_datetime(merged_new['date'])

# Merge with df
merged_new = merged_new.merge(
    holiday_halo_expanded[['location_id','sales_type_id','date','halo_type','offset_days']],
    on=['location_id','sales_type_id','date'],
    how='left'
)

# Create indicator columns
merged_new['is_holiday_halo'] = merged_new['holiday'].notna().astype(int)   # 1 if inside any halo window
merged_new['halo_offset_days'] = merged_new['offset_days'].fillna(0)        # relative distance from holiday
merged_new['halo_type'] = merged_new['halo_type'].fillna('none')

In [ ]:
# Save the updated dataframe with holiday halo features
merged_new.to_csv("daily_sales_with_holiday_halo.csv", index=False)

print("✅ File saved as daily_sales_with_holiday_halo.csv")

✅ File saved as daily_sales_with_holiday_halo.csv


In [ ]:
# 1. Calculate Daily Sales
# Include department_id in the groupby and dropna=False to include NaN department_id as a category
daily_sales_sum = merged_new.groupby(['date', 'location_id', 'sales_type_id', 'department_id'], dropna=False)['y'].transform('sum')

# Add the daily sales sum as a new column
merged_new['daily_total_sales'] = daily_sales_sum

# 2. Calculate 15-minute Percentage
# Handle potential division by zero by using .fillna(0) on the daily_total_sales
# or by using a small epsilon, but given the requirement for 0 sales days to be 0%,
# a direct division combined with fillna(0) for NaNs from division by zero is appropriate.
# However, transform('sum') will return 0 for groups with all NaNs, so direct division is fine.
# We will fill potential NaNs arising from dividing by 0 with 0.
merged_new['y_percentage_15min'] = (merged_new['y'] / merged_new['daily_total_sales']).fillna(0)

# 3. Verify Results
print("DataFrame head with daily sales and 15-minute percentage:")
display(merged_new.head())

print("\nChecking for rows where daily_total_sales is 0:")
display(merged_new[merged_new['daily_total_sales'] == 0].head())

print("\nChecking for NaN values in y_percentage_15min:")
print(merged_new['y_percentage_15min'].isnull().sum())

DataFrame head with daily sales and 15-minute percentage:


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y,real_feel,precipitation,snow,...,time,opening_time,closing_time,is_open,halo_type,offset_days,is_holiday_halo,halo_offset_days,daily_total_sales,y_percentage_15min
0,2024-01-01 00:00:00,45093,45287,NaN,46113,T7Z 2W7,22.5,0.71,0.0,0.0,...,00:00:00,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.003645
1,2024-01-01 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,...,00:15:00,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000
2,2024-01-01 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,...,00:30:00,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000
3,2024-01-01 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.0,0.71,0.0,0.0,...,00:45:00,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000
4,2024-01-01 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.49,0.0,0.0,...,01:00:00,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000



Checking for rows where daily_total_sales is 0:


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y,real_feel,precipitation,snow,...,time,opening_time,closing_time,is_open,halo_type,offset_days,is_holiday_halo,halo_offset_days,daily_total_sales,y_percentage_15min
34464,2024-12-25 00:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.56,0.0,0.0,...,00:00:00,11:00:00,00:00:00,0,none,NaN,1,0.0,0.0,0.0
34465,2024-12-25 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.56,0.0,0.0,...,00:15:00,11:00:00,00:00:00,0,none,NaN,1,0.0,0.0,0.0
34466,2024-12-25 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.56,0.0,0.0,...,00:30:00,11:00:00,00:00:00,0,none,NaN,1,0.0,0.0,0.0
34467,2024-12-25 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.56,0.0,0.0,...,00:45:00,11:00:00,00:00:00,0,none,NaN,1,0.0,0.0,0.0
34468,2024-12-25 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.0,-0.64,0.0,0.0,...,01:00:00,11:00:00,00:00:00,0,none,NaN,1,0.0,0.0,0.0



Checking for NaN values in y_percentage_15min:
0


In [ ]:
# Save the updated DataFrame
merged_new.to_csv('daily_sales_with_percentage.csv', index=False)

print("✅ Updated DataFrame saved to 'daily_sales_with_percentage.csv'")

✅ Updated DataFrame saved to 'daily_sales_with_percentage.csv'


In [ ]:
 import pandas as pd
from google.colab import files

# Upload the file
uploaded = files.upload()

# Load the correct file (based on the uploaded name)
daily_sales_with_percentage = pd.read_csv("daily_sales_with_percentage.csv", parse_dates=['ds'])

daily_sales_with_percentage.head()

Saving daily_sales_with_percentage.csv to daily_sales_with_percentage.csv


,ds,location_id,sales_type_id,department_id,y,corporation_id,postal_code,real_feel,precipitation,snow,...,time,opening_time,closing_time,is_open,halo_type,offset_days,is_holiday_halo,halo_offset_days,daily_total_sales,y_percentage_15min
0,2024-01-01 00:00:00,45093,45287,NaN,22.50,46113,T7Z 2W7,0.71,0.0,0.0,...,00:00:00,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.003645
1,2024-01-01 10:45:00,121766,244513,22316.0,5.49,78415,74429,0.08,0.0,0.0,...,10:45:00,10:00:00,22:00:00,1,holiday,0.0,1,0.0,1944.99,0.002823
2,2024-01-01 11:00:00,121752,244513,22315.0,15.98,78415,74063,0.01,0.0,0.0,...,11:00:00,10:00:00,22:00:00,1,holiday,0.0,1,0.0,1360.08,0.011749
3,2024-01-01 11:15:00,121766,244513,22313.0,0.67,78415,74429,0.15,0.0,0.0,...,11:15:00,10:00:00,22:00:00,1,holiday,0.0,1,0.0,541.49,0.001237
4,2024-01-01 11:30:00,121766,244513,22316.0,21.68,78415,74429,0.15,0.0,0.0,...,11:30:00,10:00:00,22:00:00,1,holiday,0.0,1,0.0,1944.99,0.011147


In [ ]:
daily_sales_with_percentage=merged_new.copy()

In [ ]:
### Checks

In [ ]:
# Ensure 'ds' is datetime before filtering
daily_sales_with_percentage['ds'] = pd.to_datetime(daily_sales_with_percentage['ds'])

# Filter for location_id = 45093 and January 1st, 2024
location_45093_jan_1_2024 = daily_sales_with_percentage[
    (daily_sales_with_percentage['location_id'] == 45093) &
    (daily_sales_with_percentage['ds'].dt.date == pd.to_datetime('2024-01-01').date())
].copy()

print("Data for location_id = 45093 on January 1st, 2024:")
display(location_45093_jan_1_2024)

Data for location_id = 45093 on January 1st, 2024:


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y,real_feel,precipitation,snow,...,time,opening_time,closing_time,is_open,halo_type,offset_days,is_holiday_halo,halo_offset_days,daily_total_sales,y_percentage_15min
0,2024-01-01 00:00:00,45093,45287,NaN,46113,T7Z 2W7,22.50,0.71,0.0,0.0,...,00:00:00,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.003645
1,2024-01-01 00:15:00,45093,45287,NaN,46113,T7Z 2W7,0.00,0.71,0.0,0.0,...,00:15:00,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000
2,2024-01-01 00:30:00,45093,45287,NaN,46113,T7Z 2W7,0.00,0.71,0.0,0.0,...,00:30:00,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000
3,2024-01-01 00:45:00,45093,45287,NaN,46113,T7Z 2W7,0.00,0.71,0.0,0.0,...,00:45:00,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000
4,2024-01-01 01:00:00,45093,45287,NaN,46113,T7Z 2W7,0.00,-0.49,0.0,0.0,...,01:00:00,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2024-01-01 22:45:00,45093,45287,NaN,46113,T7Z 2W7,0.00,-6.19,0.0,0.0,...,22:45:00,11:00:00,00:00:00,1,holiday,0.0,1,0.0,6173.51,0.000000
92,2024-01-01 23:00:00,45093,45287,NaN,46113,T7Z 2W7,0.00,-6.63,0.0,0.0,...,23:00:00,11:00:00,00:00:00,1,holiday,0.0,1,0.0,6173.51,0.000000
93,2024-01-01 23:15:00,45093,45287,NaN,46113,T7Z 2W7,27.25,-6.63,0.0,0.0,...,23:15:00,11:00:00,00:00:00,1,holiday,0.0,1,0.0,6173.51,0.004414
94,2024-01-01 23:30:00,45093,45287,NaN,46113,T7Z 2W7,0.00,-6.63,0.0,0.0,...,23:30:00,11:00:00,00:00:00,1,holiday,0.0,1,0.0,6173.51,0.000000


In [ ]:
# Ensure 'ds' is datetime before filtering
daily_sales_with_percentage['ds'] = pd.to_datetime(daily_sales_with_percentage['ds'])

# Filter for location_id = 121766 and January 1st, 2024
location_121766_jan_1_2024 = daily_sales_with_percentage[
    (daily_sales_with_percentage['location_id'] == 121766) &
    (daily_sales_with_percentage['ds'].dt.date == pd.to_datetime('2024-01-01').date())
].copy()

print("Data for location_id = 121766 on January 1st, 2024:")
display(location_121766_jan_1_2024)

Data for location_id = 121766 on January 1st, 2024:


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y,real_feel,precipitation,snow,...,time,opening_time,closing_time,is_open,halo_type,offset_days,is_holiday_halo,halo_offset_days,daily_total_sales,y_percentage_15min
210820,2024-01-01 00:00:00,121766,244513,22313.0,78415,74429,0.0,0.55,0.0,0.0,...,00:00:00,10:00:00,22:00:00,0,holiday,0.0,1,0.0,541.49,0.0
210821,2024-01-01 00:15:00,121766,244513,22313.0,78415,74429,0.0,0.55,0.0,0.0,...,00:15:00,10:00:00,22:00:00,0,holiday,0.0,1,0.0,541.49,0.0
210822,2024-01-01 00:30:00,121766,244513,22313.0,78415,74429,0.0,0.55,0.0,0.0,...,00:30:00,10:00:00,22:00:00,0,holiday,0.0,1,0.0,541.49,0.0
210823,2024-01-01 00:45:00,121766,244513,22313.0,78415,74429,0.0,0.55,0.0,0.0,...,00:45:00,10:00:00,22:00:00,0,holiday,0.0,1,0.0,541.49,0.0
210824,2024-01-01 01:00:00,121766,244513,22313.0,78415,74429,0.0,-0.91,0.0,0.0,...,01:00:00,10:00:00,22:00:00,0,holiday,0.0,1,0.0,541.49,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316321,2024-01-01 22:45:00,121766,244513,22316.0,78415,74429,0.0,-3.70,0.0,0.0,...,22:45:00,10:00:00,22:00:00,0,holiday,0.0,1,0.0,1944.99,0.0
316322,2024-01-01 23:00:00,121766,244513,22316.0,78415,74429,0.0,-3.85,0.0,0.0,...,23:00:00,10:00:00,22:00:00,0,holiday,0.0,1,0.0,1944.99,0.0
316323,2024-01-01 23:15:00,121766,244513,22316.0,78415,74429,0.0,-3.85,0.0,0.0,...,23:15:00,10:00:00,22:00:00,0,holiday,0.0,1,0.0,1944.99,0.0
316324,2024-01-01 23:30:00,121766,244513,22316.0,78415,74429,0.0,-3.85,0.0,0.0,...,23:30:00,10:00:00,22:00:00,0,holiday,0.0,1,0.0,1944.99,0.0


In [ ]:
filtered_actuals_15min = filtered_actuals_15min.sort_values(by='ds').reset_index(drop=True)

print("DataFrame sorted by 'ds' (date and time):")
display(filtered_actuals_15min.head())
display(filtered_actuals_15min.tail())

DataFrame sorted by 'ds' (date and time):


,ds,location_id,sales_type_id,department_id,y
0,2024-01-01 00:00:00,45093,45287,NaN,22.50
1,2024-01-01 10:45:00,121766,244513,22316.0,5.49
2,2024-01-01 11:00:00,121752,244513,22315.0,15.98
3,2024-01-01 11:15:00,121766,244513,22313.0,0.67
4,2024-01-01 11:30:00,121766,244513,22316.0,21.68


,ds,location_id,sales_type_id,department_id,y
75561,2024-12-31 22:00:00,121766,244513,22316.0,4.74
75562,2024-12-31 22:15:00,45093,45287,NaN,119.75
75563,2024-12-31 22:30:00,45093,45287,NaN,18.00
75564,2024-12-31 23:00:00,45093,45287,NaN,303.74
75565,2024-12-31 23:45:00,45093,45287,NaN,127.25


In [ ]:
filtered_actuals_15min_2025 = filtered_actuals_15min_2025.sort_values(by='ds').reset_index(drop=True)

print("DataFrame sorted by 'ds' (date and time):")
display(filtered_actuals_15min_2025.head())
display(filtered_actuals_15min_2025.tail())

DataFrame sorted by 'ds' (date and time):


,ds,location_id,sales_type_id,department_id,y
0,2025-01-01 00:30:00,121766,244513,22313.0,0.00
1,2025-01-01 10:45:00,121766,244513,22316.0,47.94
2,2025-01-01 10:45:00,121752,244513,22315.0,56.95
3,2025-01-01 11:00:00,121766,244513,22316.0,40.80
4,2025-01-01 11:15:00,121766,244513,22316.0,39.83


,ds,location_id,sales_type_id,department_id,y
37568,2025-07-02 14:15:00,121752,244513,22313.0,25.48
37569,2025-07-02 14:15:00,121752,244513,22316.0,8.54
37570,2025-07-02 14:30:00,121752,244513,22313.0,16.23
37571,2025-07-02 14:45:00,121752,244513,22316.0,7.99
37572,2025-07-02 15:00:00,121752,244513,22316.0,12.49


In [ ]:
# Get distinct location_id values from filtered_actuals_15min
distinct_location_ids = filtered_actuals_15min_2025['location_id'].unique()

print("Distinct location_id values in filtered_actuals_15min:")
display(distinct_location_ids)

Distinct location_id values in filtered_actuals_15min:


array([121766, 121752,  45093])

In [ ]:
### Can ignore the following as these are checks

In [ ]:
# Save the DataFrame for location_id 45093 on Jan 1st, 2024 to a CSV file
location_45093_jan_1_2024.to_csv('location_45093_jan_1_2024.csv', index=False)

print("DataFrame for location_id 45093 on Jan 1st, 2024 saved to 'location_45093_jan_1_2024.csv'")

DataFrame for location_id 45093 on Jan 1st, 2024 saved to 'location_45093_jan_1_2024.csv'


In [ ]:
# Save the DataFrame for location_id 121766 on Jan 1st, 2024 to a CSV file
location_121766_jan_1_2024.to_csv('location_121766_jan_1_2024.csv', index=False)

print("DataFrame for location_id 121766 on Jan 1st, 2024 saved to 'location_121766_jan_1_2024.csv'")

DataFrame for location_id 121766 on Jan 1st, 2024 saved to 'location_121766_jan_1_2024.csv'
